# Parsing Toxic Data
We have a 2 GB file called basic_data_files.csv that has a rows with many columns that we need to break down into 

Each Basic Data File contains 108 data fields, which generally represent these categories:
* Facility Name, Address, Latitude & Longitude Coordinates, SIC or NAICS codes and Industry Sector Codes
* Chemical Identification and Classification Information
* On-site Release Quantities
* Publicly Owned Treatment Works (POTW) Transfer Quantities
* Off-site Transfer Quantities for Release/Disposal and Further Waste Management
* Summary Pollution Prevention Quantities (Section 8 of the Form R)

## Relations
We have the following tables that supporting the following textual description. Companies manage Facilities. Facilities release chemicals. Facilities may release chemicals repeatedly. Facilities may release more than one chemical.

* Company(name:string)
* Facility(company:references, name:string, address:string, latitude:float, longitude:float, SIC_code:string, NAICS_code:string, industry_sector_code:string)
* Chemical(name:string, clear_air_act_chemical:boolean, is_metal:boolean, metal_category:string)
* Release(facility:references, chemical:references, date:datetime, method:string, quantity:integer, units:string)

## Data Wrangling

There are significantly more columns in the basic data files than are captured in these relations, yet this schema is still too complicated. Releases will be queried and compared by time of release as well as quantity. Our storage of quantity as an integer with a string unit is problematic for comparison. The assumption is that runtime computation will compute the comparisons unless we store everything in the same units.

## Goals

* Acquire all of the information for the tables as described
* Convert all of the quantities to a single unit 
* Change the Release table according to the unit
* Insert recrods into the MySQL for each table


In [45]:
import pandas as pd

files = pd.read_csv('basic_data_files.csv', chunksize=8192, nrows=10)

companies = set()
for df in files:
    names = df['PARENT_COMPANY_NAME']
    names.fillna(0,inplace=True)
    for company in names:
        companies.add(company)

companies.remove(0)
companies



{'ACUSHNET HOLDINGS CORP',
 'CARGILL INC',
 'GENERAL MOTORS LLC',
 'HANNA STEEL CORP',
 'KICE INDUSTRIES INC',
 'THE CLOROX CO',
 'US DEPARTMENT OF THE TREASURY'}